In [1]:
import implicit
from scipy.sparse import coo_matrix

import pandas as pd
import ml_metrics as metrics

import lightgbm as lgb
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import random
from collections import Counter


# inference

In [47]:
user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')

user_hist_df['ts'] = pd.to_datetime(user_hist_df['ts'])
user_hist_df['month'] = user_hist_df['ts'].dt.month

user_hist_df = user_hist_df[user_hist_df['episode_id'] == 0]
train = train = user_hist_df[~user_hist_df['month'].isin([10, 11])]

In [48]:
train['event'] = 1
train = train[['user_id', 'movie_id', 'event']].drop_duplicates()

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
user_encode = {u: i for i, u in enumerate(train['user_id'].unique())}
item_encode = {u: i for i, u in enumerate(train['movie_id'].unique())}

user_decode = {v: k for k, v in user_encode.items()}
item_decode = {v: k for k, v in item_encode.items()}

n_users, n_items = len(user_encode), len(item_encode)

train['user_id'] = train['user_id'].apply(lambda x: user_encode[x])
train['movie_id'] = train['movie_id'].apply(lambda x: item_encode[x])

In [14]:
item_user = coo_matrix(
    (train['event'], (train['user_id'], train['movie_id'])), shape=(n_users, n_items))

user_item = coo_matrix(
    (train['event'], (train['movie_id'], train['user_id'])), shape=(n_items, n_users))

In [15]:
user_item.shape

(3497, 2022)

In [16]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=5, iterations= 15)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(user_item, show_progress=False)

In [17]:
movies_database = pd.read_csv('../data/raw/movies.csv')
useless_movies = movies_database[(movies_database['year'] < 2010) & 
                                 (~movies_database['id'].isin(list(item_encode.keys()))) &
                                 (movies_database['imdb_rating'] < 6)]['id'].values

useless_movies = [m for m in useless_movies if m in item_decode]

In [18]:
# als
res = model.recommend_all(item_user, N = 100, filter_items = useless_movies, show_progress=False)

In [19]:
recom = {}

for ind, rec in enumerate(res):
    recom[user_decode[ind]] = [item_decode[x] for x in rec]

# LGBM

In [11]:
user_features = model.user_factors

get item features

In [12]:
movies_database = pd.read_csv('../data/raw/movies.csv')

# movies_database = movies_database[movies_database['id'].isin(list(item_encode.keys()))]
item_features = movies_database[['id', 'year', 'genres', 'imdb_rating', 'tmdb_rating']]

def convert_year(val):
    if val < 2000:
        cat = 0
    elif 2000<=val < 2010:
        cat = 1
    else:
        cat = 2
    return cat

item_features['year'] = item_features['year'].apply(convert_year)

user_hist_df = pd.read_csv('../data/raw/movies_dataset_10 months.csv')
movies = user_hist_df[user_hist_df['episode_id'] == 0]['movie_id'].unique()

item_features['movie'] = item_features['id'].apply(lambda x: 1 if x in movies else 0 )

test = item_features['genres'].fillna('hz').str.split(',')

mlb = MultiLabelBinarizer()
res = pd.DataFrame(mlb.fit_transform(test),
                   columns=mlb.classes_,
                   index=test.index)

to_drop = ['Art House', 'Documentary', 'Family','Musical', 'Quarantine Info', 'hz', 'War']
res = res.drop(to_drop, 1)

item_features = pd.concat([item_features, res], 1).drop('genres', 1)

item_features = item_features.fillna(item_features.mean())

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
item_features = item_features.drop('movie', 1)

In [14]:
user_features_df = pd.DataFrame(user_features)
user_features_df['user_id'] = user_encode.values()

In [15]:
rank_df = pd.merge(train, user_features_df, how='left', on='user_id')

In [16]:
rank_df = pd.merge(rank_df, item_features, how = 'left', 
                   left_on ='movie_id', 
                   right_on='id')

In [17]:
rank_df.shape

(154776, 34)

In [18]:
rank_df = rank_df.dropna()

generate negative examples

In [19]:
user_hist = train.groupby('user_id')['movie_id'].apply(set)
users_to_predict = train['user_id'].unique()

In [20]:
movie_genres = movies_database[['id', 'genres']]
movie_genres['genres'] = movie_genres['genres'].str.split(',')
#movie_genres['id'] = movie_genres['id'].apply(lambda x: item_encode.get(x, x + 1000000000) )

movie_genres_dict = {}

for i in movie_genres.iterrows():
    movie_genres_dict[i[1][0]] = i[1][1]

/Users/danil/Documents/github/sweet_RS/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
movie_genres_dict = {k: v for k, v in movie_genres_dict.items() if isinstance(v, list)}

вот тут еще можно поиграться с кол вом жанров которые нужно учесть

In [22]:
user_pref = {}
for user in users_to_predict:
    #print(user)
    user_pref[user] = {}
    #print(user_hist[user])
    for film in user_hist[user]:
        film = item_decode[film]
        if movie_genres_dict.get(film):
            for genre in movie_genres_dict[film]:
                if user_pref[user].get(genre):
                    user_pref[user][genre] += 1
                else:
                    user_pref[user][genre] = 1
                    
    user_pref[user] = [key for key, val in Counter(user_pref[user]).most_common(5)]                



In [23]:
neg_examples = 50

In [24]:
user_negative = {}

for user in user_pref:
    negative = []
    current_user_pref = set(user_pref[user])
    
    num = 0
    while num < neg_examples:
        random_film = random.sample(list(movie_genres_dict), 1)[0]
        curr_film_genres = set(movie_genres_dict[random_film])
        if len(current_user_pref.intersection(curr_film_genres)) == 0:
            negative.append(random_film)
            num += 1
            
    user_negative[user] = negative
      

In [25]:
users = []
films = []


for user in user_negative:
    users += [user] * neg_examples
    films += user_negative[user]
    
    
negative_df = pd.DataFrame()
negative_df['user_id'] = users
negative_df['movie_id'] = films

In [26]:
negative_df = pd.merge(negative_df, user_features_df, how='left', on='user_id')

In [27]:
negative_df = pd.merge(negative_df, item_features, how = 'left', 
                       left_on ='movie_id', 
                       right_on='id')
negative_df['event'] = 0

In [28]:
negative_df.shape

(101100, 34)

In [29]:
rank_df.shape

(153711, 34)

In [30]:
#X = pd.concat([negative_df[:10000], rank_df[:10000]], 0)
X = pd.concat([negative_df[:], rank_df[:]], 0)

In [31]:
X.iloc[:, 11:] = X.iloc[:, 11:].astype('int8')
X.iloc[:, 2:7] = X.iloc[:, 2:7].astype('float16')

In [6]:
import implicit
from scipy.sparse import coo_matrix

import pandas as pd
import ml_metrics as metrics

import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import random
from collections import Counter

import sys

PATH = '/Users/danil/Documents/github/sweet_RS/'
sys.path.append(str(PATH))

from src.utils import save_to_pickle, read_pickle

In [34]:
save_to_pickle(X, '../data/processed/X_for_2lvl.pickle')
save_to_pickle(user_encode, '../data/processed/user_encode.pickle')
save_to_pickle(user_features_df, '../data/processed/user_features_df.pickle')
save_to_pickle(item_features, '../data/processed/item_features.pickle')
save_to_pickle(user_decode, '../data/processed/user_decode.pickle')
save_to_pickle(recom, '../data/processed/recom.pickle')

In [7]:
X = read_pickle('../data/processed/X_for_2lvl.pickle')
user_encode = read_pickle('../data/processed/user_encode.pickle')
user_features_df = read_pickle('../data/processed/user_features_df.pickle')
item_features = read_pickle('../data/processed/item_features.pickle')
user_decode = read_pickle('../data/processed/user_decode.pickle')
recom = read_pickle('../data/processed/recom.pickle')

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.drop(['user_id', 'movie_id', 'id', 'event'], 1), 
                                                    X['event'], test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [16]:
y_train = y_train[:]
X_train = X_train[:50000]

In [21]:
query_train = [X_train.shape[0]]
query_val = [X_val.shape[0]]
query_val = [10000]
query_test = [X_test.shape[0]]

In [30]:
del gbm

In [6]:
gbm = lgb.LGBMRanker(max_depth = 5, n_estimators = 100, random_state = 42, 
                    n_jobs=-1)

In [18]:
model = xgb.XGBRanker()

In [23]:
model.fit(X_train, y_train, group=query_train,
        eval_set=[(X_val[:10000], y_val[:10000])], eval_group=[query_val],
          early_stopping_rounds=50)

XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
          importance_type='gain', interaction_constraints='',
          learning_rate=0.300000012, max_delta_step=0, max_depth=6,
          min_child_weight=1, missing=nan, monotone_constraints='()',
          n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
          tree_method='exact', validate_parameters=1, verbosity=None)

In [35]:
model.fit(X_train, y_train, group=query_train)

XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
          importance_type='gain', interaction_constraints='',
          learning_rate=0.300000012, max_delta_step=0, max_depth=6,
          min_child_weight=1, missing=nan, monotone_constraints='()',
          n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=1,
          tree_method='exact', validate_parameters=1, verbosity=None)

validate

In [36]:
users = []
films = []


for user in recom:
    users += [user_encode[user]] * 100
    films += recom[user]
    
validate_df = pd.DataFrame()
validate_df['user_id'] = users
validate_df['movie_id'] = films

In [37]:
validate_df = pd.merge(validate_df, user_features_df, how='left', on='user_id')

validate_df = pd.merge(validate_df, item_features, how = 'left', 
                       left_on ='movie_id', 
                       right_on='id')


validate_df = validate_df.dropna()


In [38]:
test_pred = model.predict(validate_df.drop(['user_id', 'movie_id', 'id'], 1))
validate_df['pred'] = test_pred

In [39]:
g = validate_df.groupby(['user_id']).apply(lambda x: x.nlargest(5,['pred'])).reset_index(drop=True)


In [30]:
recoms_wl = g.groupby('user_id')['movie_id'].apply(list)

In [43]:
recoms_to_save = {user_decode[k]: v for k, v in recoms_wl.items()}

In [32]:
save_to_pickle(recoms_to_save, '../data/processed/2_lvl_xgbm.pickle')

In [33]:
len(recoms_to_save)

2022

In [41]:
recoms_wl

user_id
0         [4644, 18413, 10544, 1176, 19353]
1       [17261, 15403, 16861, 13889, 18652]
2       [19252, 17261, 16476, 16032, 17218]
3         [2973, 19252, 17261, 4386, 19852]
4        [19207, 17261, 16693, 1989, 10920]
                       ...                 
2017     [9335, 17261, 10920, 19207, 18576]
2018     [9335, 18586, 19353, 18576, 17364]
2019     [18586, 9256, 10462, 10389, 19353]
2020       [3612, 2776, 10680, 6128, 15173]
2021     [19353, 17364, 18576, 16601, 2219]
Name: movie_id, Length: 2022, dtype: object